Este es un "notebook" (cuaderno digital) para mostrar el efecto de la regularización de Ridge y Lasso.

Acompaña al Capítulo 5 del libro (3 de 5).

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
from sklearn import linear_model

In [ ]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
from sklearn.model_selection import cross_validate, KFold

In [ ]:
np.random.seed(16) # establecer semilla para la reproducibilidad de resultados

x1 = np.arange(100) 

x2 = np.linspace(0,1,100)

x3 = np.logspace(2,3,num=100) 

ypb = 3*x1 + 0.5*x2 + 15*x3 + 3 + 5*(np.random.poisson(3*x1 + 0.5*x2 + 15*x3,100)-(3*x1 + 0.5*x2 + 15*x3)) 
                                                    #generar algunos datos con dispersión siguiendo la distribución de Poisson
                                                    #con el valor exp = y del modelo lineal, centrado alrededor de 0

In [ ]:
xb = np.vstack((x1,x2,x3)).T

In [ ]:
xb.shape

### Agregar características correlacionadas (transformación polinomial)

In [ ]:
poly = PolynomialFeatures(2, include_bias=False)

In [ ]:
new_xb = poly.fit_transform(xb)

### Registro de Aprendizaje

¿Cuántas características tendrá el conjunto de datos transformado? Primero podemos pensarlo y luego usar el código para averiguarlo.    

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
new_xb.shape

mostrará que hay 9 características (las originales, más todas las combinaciones monomiales de x1, x2 y x3 hasta el grado 1, por ejemplo x1^2, x1 x2...)
```
</p>
</details>

### Comencemos con la regresión de Ridge y ajustemos alfa usando validación cruzada.

(tener en cuenta lo que sucede si se repite varias veces sin arreglar la semilla aleatoria).

In [ ]:
MSE = []

for alpha in np.logspace(-6,6,13):

    model_reg = make_pipeline(StandardScaler(), Ridge(alpha = alpha, max_iter=10000)) #la normalización ayuda a la convergencia

    scores = cross_validate(model_reg, new_xb, ypb, cv = KFold(n_splits=5, shuffle=True, random_state = 1), scoring = 'neg_mean_squared_error')

    print(alpha, np.round(-np.mean(scores['test_score']))) #¡Esto es solo ECM!
    
    MSE.append(-np.mean(scores['test_score']))

print('Mejor alpha:', np.logspace(-6,6,13)[np.argmin(MSE)])

# ¡También hay un instrumento incorporado para esto!

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
regm = make_pipeline(StandardScaler(with_mean=False),
                     RidgeCV(alphas=np.logspace(-6,6,13), \
            cv = KFold(n_splits=5, shuffle=True, random_state=1),\
             scoring = 'neg_mean_squared_error'))

regm.fit(new_xb,ypb); 

print('El mejor alpha es', regm[1].alpha_)

### Podemos comparar los coeficientes del modelo lineal para diferentes cantidades de regularización.

#### Elijamos alfa = 1000.

In [ ]:
#Nota: la notación cambió en sklearn 1.2 y superior;
#Para reproducir los resultados, necesitamos usar alpha = alpha * n_samples

newmodel = make_pipeline(StandardScaler(with_mean=False), Ridge(alpha=1000*new_xb.shape[0]))

newmodel.fit(new_xb,ypb)

coef_alpha_1000 = np.hstack([newmodel[1].coef_/newmodel[0].scale_, newmodel[1].intercept_])
                         
print(coef_alpha_1000)

### Registro de aprendizaje

¿Serán los coeficientes mayores o menores, en comparación con el caso con alfa = 1000?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Los coeficientes serán más grandes porque usamos una regularización más débil y el efecto de la regularización en un modelo lineal es mantener los coeficientes pequeños.
```
</p>
</details>

In [ ]:
#Nota: la notación cambió en sklearn 1.2 y superior;
#Para reproducir los resultados, necesitamos usar alpha = alpha * n_samples

newmodel = make_pipeline(StandardScaler(with_mean=False), Ridge(alpha=1*new_xb.shape[0]))

newmodel.fit(new_xb,ypb)

coef_alpha_1 = np.hstack([newmodel[1].coef_/newmodel[0].scale_, newmodel[1].intercept_])
                         
print(coef_alpha_1)

#### A continuación, usamos un truco para obtener coeficientes para alfa "cero" (sin regularización); También podríamos haber usado LinearRegressor.

In [ ]:
newmodel = make_pipeline(StandardScaler(with_mean=False), Ridge(alpha=1e-11*new_xb.shape[0]))

newmodel.fit(new_xb,ypb)

coef_alpha_noreg = np.hstack([newmodel[1].coef_/newmodel[0].scale_, newmodel[1].intercept_])
                         
print(coef_alpha_noreg)

Comparación con el mismo procedimiento para el modelo lineal (sin regularización):

In [ ]:
lmodel = make_pipeline(StandardScaler(with_mean=False),linear_model.LinearRegression())
lmodel.fit(new_xb,ypb)
print(lmodel[1].coef_/lmodel[0].scale_, lmodel[1].intercept_)

### Podemos comparar los coeficientes para los tres casos.

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(np.arange(10)-0.2, np.abs(coef_alpha_1000), color = 'maroon',width=0.05, label = 'Ridge, alpha = 1000')
plt.bar(np.arange(10)-0.1, np.abs(coef_alpha_1), color = 'orangered',width=0.05, label = 'Ridge, alpha = 1.0')
plt.bar(range(10), np.abs(coef_alpha_noreg), color = 'grey',width=0.05, label = 'Lineal (sin regularización)')
plt.yscale('log')

plt.xticks(np.arange(10), ['1','2', '3','4','5','6','7','8','9', 'Intercepto'])  # Poner etiquetas de texto.

plt.xlabel('Característica',fontsize=14)

plt.ylabel('Coeficientes (valor absoluto)',fontsize=14)

plt.legend(fontsize=13);

Echemos un vistazo a LASSO.

In [ ]:
from sklearn.linear_model import Lasso, LassoCV 

In [ ]:
#Nota: la notación cambió en sklearn 1.2 y superior;
#Para reproducir los resultados, necesitamos usar alpha = alpha * sqrt(n_samples)

#Nota: ¡LassoCV reordena los alfas en ORDEN DESCENDENTE! Las notas se desordenarán a menos que uses el objeto model.alphas_

model = make_pipeline(StandardScaler(with_mean = False), \
                      LassoCV(alphas = np.logspace(-1,4,6), 
                        cv = KFold(n_splits=5, shuffle=True, random_state=1), \
              max_iter = 10000000, tol = 1e-6))

model.fit(new_xb,ypb)

print('Alphas', model[1].alphas_)

print('Mejor alpha:', model[1].alpha_)

for i, alpha in enumerate(model[1].alphas_):
    print('Nota para alpha', alpha, np.mean(model[1].mse_path_[i,:])) #para cada alfa (fila), 10 estimaciones cv de ECM

#### <font color = 'red'> Nota: los primeros problemas de reproducibilidad se resolvieron ajustando la tolerancia a un valor pequeño (¡gracias a Joel Zinn!). </font>

Veamos los coeficientes para alfa = 1000 y alfa = 1. La regularización de lazo tiende a inducir coeficientes dispersos, ¡así que podemos comprobar que eso es cierto!

In [ ]:
L1000 = make_pipeline(StandardScaler(with_mean=False), Lasso(alpha = np.sqrt(new_xb.shape[0])*1000, max_iter = 1000000, tol = 0.005))

L1000.fit(new_xb, ypb)

coef_L1000 =  np.hstack([L1000[1].coef_, L1000[1].intercept_])

print(coef_L1000)

### Registro de Aprendizaje

¿Deberíamos preocuparnos porque:

1) ¿La intersección se ha vuelto muy grande?

2) ¿Todos los coeficientes han desaparecido?

<br>

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
1) No realmente, porque la interceptación está excluida del proceso de regularización;

2) Solo si esto sucede cuando se usa el mismo código y un valor mucho más pequeño para alfa :)
```
</p>
</details>

In [ ]:
L1 = make_pipeline(StandardScaler(with_mean=False), Lasso(alpha = np.sqrt(new_xb.shape[0])*1, max_iter = 1000000, tol = 0.005))

L1.fit(new_xb, ypb)

coef_L1 =  np.hstack([L1[1].coef_/L1[0].scale_, L1[1].intercept_])

print(coef_L1)

### Finalmente, podemos graficar todos los coeficientes juntos.

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(np.arange(10)-0.2, np.abs(coef_alpha_1000), color = 'maroon',width=0.05, label = 'Ridge, alpha = 1000')
plt.bar(np.arange(10)-0.1, np.abs(coef_alpha_1), color = 'orangered',width=0.05, label = 'Ridge, alpha = 1.0')
plt.bar(range(10), np.abs(coef_alpha_noreg), color = 'grey',width=0.05, label = 'Lineal (no regularization)')
plt.bar(np.arange(10)+0.1, np.abs(coef_L1), color = 'tab:cyan',width=0.05, label = 'Lasso, alpha = 1.0')
plt.bar(np.arange(10)+0.2, np.abs(coef_L1000), color = 'tab:blue', width=0.05, label = 'Lasso, alpha = 1000')

plt.yscale('log')

plt.xticks(np.arange(10), ['1','2', '3','4','5','6','7','8','9', 'Intercepto'])  # Establecer etiquetas de texto.

plt.xlabel('Característica',fontsize=14)

plt.ylabel('Coeficientes (valor absoluto)',fontsize=14)

plt.legend(fontsize=13, bbox_to_anchor=(1.05, 1));
